In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

churn = pd.read_excel('Customer-Churn-Records.xlsx')

An uncleaned version of the Customer-Churn-Records dataset has also been loaded as the data frame `churn`.

Understanding the overall structure of the dataset 
The `info()` function gives you a concise summary of the data frame, including data types, non-null counts, and memory usage. It's a great way to quickly understand the structure of your dataset, identify data types and detect null values. In this case, you can see that your data frame is `10,000` rows long and contains no `null` values:

In [2]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   RowNumber           10000 non-null  int64  
 1   CustomerId          10000 non-null  int64  
 2   Surname             10000 non-null  object 
 3   CreditScore         10000 non-null  int64  
 4   Geography           10000 non-null  object 
 5   Gender              10000 non-null  object 
 6   Age                 10000 non-null  int64  
 7   Tenure              10000 non-null  int64  
 8   Balance             10000 non-null  float64
 9   NumOfProducts       10000 non-null  int64  
 10  HasCrCard           10000 non-null  int64  
 11  IsActiveMember      10000 non-null  int64  
 12  EstimatedSalary     10000 non-null  float64
 13  Exited              10000 non-null  int64  
 14  Complain            10000 non-null  int64  
 15  Satisfaction Score  10000 non-null  int64  
 16  Card 

You can also use the `describe()` function to get some summary statistics on the numeric variables in your dataset, such as count, mean, standard deviation, min, max and percentiles:

In [3]:
churn.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Point Earned
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203800,0.204400,3.013800,606.515100
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402842,0.403283,1.405919,225.924839
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000,0.000000,1.000000,119.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000,0.000000,2.000000,410.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000,0.000000,3.000000,605.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000,0.000000,4.000000,801.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000,1.000000,5.000000,1000.000000


The `value_counts()` function can be applied to a variable and returns a list of unique values with the counts of each. It's handy for exploring the distribution of values in a column. 
 
Let’s apply it to the `'Geography'` column in the `'churn'` data frame. Here you will discover your first data quality issue!

In [4]:
churn['Geography'].value_counts()

Geography
NSW    5014
VIC    2509
QLD    2477
Name: count, dtype: int64

##  Identifying duplicates, null and missing values

One way to identify duplicates is via the `duplicated()` function. Let’s apply it to a full row using the `columns` parameter, and to the unique identifier `CustomerId`. This will give us comfort that each customer is only presented once in the dataset. The print function returns the column headings with no rows below them, because there are no duplicates to display.

In [5]:
# Check for duplicate rows
print(churn[churn.duplicated(subset=churn.columns)])
 
# Check for duplicate primary keys
print(churn[churn.duplicated(subset='CustomerId')])

Empty DataFrame
Columns: [RowNumber, CustomerId, Surname, CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary, Exited, Complain, Satisfaction Score, Card Type, Point Earned]
Index: []
Empty DataFrame
Columns: [RowNumber, CustomerId, Surname, CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary, Exited, Complain, Satisfaction Score, Card Type, Point Earned]
Index: []


## Checking whether values are within range

There are several ways to check whether values within a column are within their expected range. In this example, you will use a `Boolean series` to count the number of observations in the `CreditScore` column that fall outside the expected range of `350` to `850`.
 
Here's a breakdown of what each line does:
 
1. `min_credit, max_credit = 350, 850`: This line defines the minimum and maximum acceptable values for the credit score, which are `350` and `850` respectively.
 
2. `condition = (churn['CreditScore'] >= min_credit) & (churn['CreditScore'] <= max_credit)`: This line creates a new series where each element is either `True` or `False`. It checks each value in the `'CreditScore'` column to see if it lies within the specified range. If a value is within the range, the corresponding value in the `'condition'` series will be `True`; otherwise, it will be `False`.
 
3. `value_counts = condition.value_counts()`: This line counts the number of True and False values in the 'condition' series. The result is a series where the index is `[True, False]` and the values are the counts of how many times each appears in the `'condition'` series.
 
4. `print(value_counts)`: This line prints the counts of `True` and `False` values. The count of `True` values represents the number of records in the `'CreditScore'` column that are within the specified range, while the count of `False` values represents the number of records that are outside this range.

In [6]:
# Define acceptable range
min_credit, max_credit = 350, 850
 
# Check if all CreditScores are within range
condition = (churn['CreditScore'] >= min_credit) & (churn['CreditScore'] <= max_credit)
value_counts = condition.value_counts()
 
print(value_counts)

CreditScore
True    10000
Name: count, dtype: int64


## Identifying special characters with the ReGex module

The Python `re` module is used for working with Regular Expressions (RegEx). RegEx is a sequence of characters that forms a search pattern. This pattern can be used to search, replace or split text in strings.
 
The `re` module offers several functions to work with RegEx:
 
- `re.match()`: This function checks if the RegEx pattern matches at the beginning of a string.
- `re.search()`: This function searches the string for a match to the RegEx pattern and returns a match object if found. Unlike `re.match()`, it doesn't require the pattern to be at the beginning of the string.
- `re.findall()`: This function returns all non-overlapping matches of the RegEx pattern as a list of strings.
- `re.split()`: This function splits the string by the occurrences of the RegEx pattern.
- `re.sub()`: This function replaces occurrences of the RegEx pattern in the string with a specified replacement string.
- `re.compile()`: This function compiles a RegEx pattern into a regular expression object, which can be used for matching using its `match()` and `search()` methods, among others.


Let's use the `re.findall()` function to test whether there are any random special characters in the  `Surname` column in the `churn` data frame:


In [7]:
# Define regular special characters
pattern = r'[!@#$%^&*()_+]'
 
# Scan the 'Surname' column for special characters
for x in churn['Surname']:
    special_characters = re.findall(pattern, x)
    if special_characters:
        print(f"Special characters found in", x)

NameError: name 're' is not defined

Here's what's going on in the script:
 
1. `pattern = r'[!@#$%^&*()_+]`: This line defines a RegEx pattern that matches any of the special characters inside the square brackets. The r before the string indicates that this is a raw string, which treats backslashes as literal characters.
2. The for loop `for x in churn['Surname']`: iterates over each value in the 'Surname' column of the `'churn'` data frame.
3. `special_characters = re.findall(pattern, x)`: For each surname, the `re.findall()` function is used to find all occurrences of the defined pattern. The result is a list of all special characters found in the surname.
 
So, in essence, this code is scanning each surname in the `'Surname'` column of the `'churn'` data frame for any of the specified special characters. If a surname contains any of these special characters, it prints that surname.
 
As a result, you found one email address that had been erroneously entered into the `'Surname'` column.